In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandarallel import pandarallel
import numpy as np
import matplotlib.pyplot as plt

from microsim.population import NHANESDirectSamplePopulation

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

In [2]:
#from microsim.sim_settings import simSettings
#pandarallel.initialize(verbose=1) #microsim by default now does not initialize pandarallel
#simSettings.pandarallelFlag = True #with this flag all new population instances will be set to use pandarallel
#seedSequence = np.random.SeedSequence()
#rngStream = np.random.default_rng(seed=seedSequence)
#from microsim.bp_treatment_strategies import AddASingleBPMedTreatmentStrategy

In [3]:
%%time
popSize = 10000
nYears = 18
pop = NHANESDirectSamplePopulation(popSize, 1999)

CPU times: user 3.94 s, sys: 35.5 ms, total: 3.98 s
Wall time: 3.98 s


In [ ]:
%%time
#3m 20s
pop.advance(nYears)

In [4]:
%%time
#1m 47 s
pop.advance_parallel(nYears, None, 4)

CPU times: user 3.39 s, sys: 233 ms, total: 3.63 s
Wall time: 58.5 s


In [ ]:
#pop = NHANESDirectSamplePopulation(popSize, 2017, rng=rngStream)
#controlPop = NHANESDirectSamplePopulation(popSize, 1999, rng=rngStream)
#treatmentPop = NHANESDirectSamplePopulation(popSize, 1999, rng=rngStream)
#treatmentPop.set_bp_treatment_strategy(AddASingleBPMedTreatmentStrategy())

In [ ]:
%%time
#40min
#alive, dead = pop.advance_vectorized(18,rng=rngStream) #5m 9s - no recalibration
#aliveC, deadC = controlPop.advance_vectorized(5,rng=rngStream)
#aliveT, deadT = treatmentPop.advance_vectorized(5,rng=rngStream)

In [ ]:
#controlRisk = sum(list(map(lambda x: x.has_stroke_during_simulation(), controlPop._people)))/controlPop.n
#treatmentRisk = sum(list(map(lambda x: x.has_stroke_during_simulation(), treatmentPop._people)))/treatmentPop.n
#0.87 
#print(treatmentRisk/controlRisk)
#controlRisk = sum(list(map(lambda x: x.has_mi_during_simulation(), controlPop._people)))/controlPop.n
#treatmentRisk = sum(list(map(lambda x: x.has_mi_during_simulation(), treatmentPop._people)))/treatmentPop.n
#0.87 
#print(treatmentRisk/controlRisk)

In [ ]:
from microsim.risk_factor import DynamicRiskFactorsType
from microsim.population import microsimToNhanes

nhanesPop = pd.read_stata(microsimDir + "/microsim/data/fullyImputedDataset.dta")
nhanesPop = nhanesPop[~(nhanesPop["timeInUS"]<4)]
nhanesPop = nhanesPop[nhanesPop["year"]==2017]
nhanesPop = nhanesPop[nhanesPop["age"]>=36]

In [ ]:
nRows = round(len(DynamicRiskFactorsType)/2)
fig, ax = plt.subplots(nRows, 2, figsize=(17,15))
row=-1
for i,rf in enumerate(DynamicRiskFactorsType):
    rfList = list(map( lambda x: getattr(x, "_"+rf.value)[-1] if x.is_alive else None, pop._people))
    rfList = list(filter(lambda x: x is not None, rfList))
    rfList = list(map(lambda x: int(x) if bool(x) else x, rfList))
    if i%2==0:
        row += 1
        col = 0
    else:
        col = 1
    if rf.value in microsimToNhanes.keys():
        ax[row,col].hist([rfList, nhanesPop[microsimToNhanes[rf.value]]], bins=20, density=True)
    else:
        ax[row,col].hist(rfList, bins=20, density=True)
    ax[row,col].set_xlabel(rf.value)
    #ax[row,col].set_ylabel("probability density")
plt.suptitle("probability densities for all dynamic risk factors")
#plt.subplots_adjust(wspace=0.5, hspace=0.7)
plt.tight_layout()
plt.show()

In [ ]:
space = " "
print("                       sim mean   nha mean    sim sd     nha sd")
for i,rf in enumerate(DynamicRiskFactorsType):
    rfList = list(map( lambda x: getattr(x, "_"+rf.value)[-1] if x.is_alive else None, pop._people))
    rfList = list(filter(lambda x: x is not None, rfList))
    rfList = list(map(lambda x: int(x) if bool(x) else x, rfList))
    if rf.value in microsimToNhanes.keys():
        print(f"{rf.value:>20} {np.mean(rfList): 10.2f} {np.mean(nhanesPop[microsimToNhanes[rf.value]]): 10.2f}" + 
             f"{np.std(rfList): 10.2f} {np.std(nhanesPop[microsimToNhanes[rf.value]]): 10.2f}")
    else:
        print(f"{rf.value:>20} {np.mean(rfList): 10.2f} {space:>9} {np.std(rfList): 10.2f}")

In [ ]:
from microsim.outcome import OutcomeType
outcomes = [OutcomeType.MI, OutcomeType.STROKE, OutcomeType.DEATH, 
            OutcomeType.CARDIOVASCULAR, OutcomeType.NONCARDIOVASCULAR, OutcomeType.DEMENTIA]
standardizedRates = list(map(lambda x: pop.calculate_mean_age_sex_standardized_incidence(x, 2016), outcomes))
standardizedRatesBlack = list(map(
                           lambda x: pop.calculate_mean_age_sex_standardized_incidence(x,2016, lambda y: y._black), 
                           outcomes))
print("standardized rates (per 100,000)    all        black   ")
for i in range(len(outcomes)):
    print(f"{outcomes[i].value:>30} {standardizedRates[i]:> 10.1f} {standardizedRatesBlack[i]:> 10.1f}") 

In [ ]:
dementiaIncidentRate = pop.get_raw_incidence_by_age(OutcomeType.DEMENTIA)

In [ ]:
plt.scatter(dementiaIncidentRate.keys(), dementiaIncidentRate.values())
plt.xlabel("age")
plt.ylabel("dementia incidence rate")
plt.show()

In [ ]:
ageDementia = list(map(lambda y: (y._age[-1], len(y._outcomes[OutcomeType.DEMENTIA])>0), 
              list(filter(lambda x: x.is_alive, pop._people))))
nAlive = len(ageDementia)
ageDementia = list(filter(lambda x: x[1]==True, ageDementia))
ageDementia = [int(x[0]) for x in ageDementia]
plt.hist(ageDementia)
plt.xlabel("age")
plt.title(f"dementia cases at end of simulation ({nAlive} Person objects alive)")
plt.show()

In [ ]:
%%time
#35min for 200k
from microsim.treatment_strategy_repository import TreatmentStrategyRepository
from microsim.bp_treatment_strategies import AddASingleBPMedTreatmentStrategy
from microsim.treatment import TreatmentStrategiesType
from microsim.outcome import OutcomeType

popSize = 200000
treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)
controlPop = NHANESDirectSamplePopulation(popSize, 1999)
nYears=5
treatmentStrategies = TreatmentStrategyRepository()
treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddASingleBPMedTreatmentStrategy()
treatmentPop.advance(nYears, treatmentStrategies = treatmentStrategies)
controlPop.advance(nYears)

In [ ]:
controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                           controlPop._people)))/controlPop._n
treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                             treatmentPop._people)))/treatmentPop._n
#0.93
controlRisk, treatmentRisk, treatmentRisk/controlRisk

In [ ]:
controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                           controlPop._people)))/controlPop._n
treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                             treatmentPop._people)))/treatmentPop._n
#0.90
controlRisk, treatmentRisk, treatmentRisk/controlRisk